---
title: Predicting wine quality given chemical characteristics of the wine
author: Yixuan Gao, Bryan Lee, Wangkai Zhu, Timothy Singh
format:
    html:
        toc: true
        toc-depth: 4
    pdf:
        toc: true
        toc-depth: 4
editor: source
bibliography: references.bib
execute:
    echo: false
    warning: false
---




## Summary

This project aims to build a machine learning model to classify the quality of red wine based on its physicochemical properties. The target variable, wine quality, is scored on a discrete scale from 0 (poor quality) to 10 (high quality), presenting a multi-class classification challenge. The dataset, sourced from the UC Irvine Machine Learning Repository [@uci_wine_quality], comprises 1,599 observations with 11 continuous features such as acidity, alcohol content, and citric acid [@cortez2009modeling].

Six (6) classification algorithms were evaluated: Dummy Classifier (as a baseline), Logistic Regression, Decision Tree, K-Nearest Neighbors (KNN), Naive Bayes and Support Vector Machine with a Radial Basis Function kernel (SVM RBF). 5-fold cross-validation was used with training data to find the best classification algorithm, based on accuracy, which was the SVM RBF model. Hyperparameter tuning was used to optimize this model and assess its generalization performance. The `C`, `gamma` and `decision_function_shape` hyperparameters on SVC were tuned using the `RandomizedSearchCV()` from `sklearn`. The model with hyperparameters that gave the best accuracy was selected for deployment on the test set. This model gave an accuracy on the testing set of around 0.6.

By leveraging machine learning, this project seeks to provide a systematic and measurable way to predict wine quality, aiding manufacturers and suppliers in assessing product value based on its chemical properties.



## Introduction

Red wine has been a cultural and economic staple since ancient times, originating from early civilizations like the Greeks and evolving into a global industry valued at approximately 109.5 billion USD. Despite its widespread availability, differentiating between high and low-quality wines remains a challenge for most consumers. Traditionally, this task has relied on the expertise of sommeliers, whose judgments are often subjective.

This project aims to bridge the gap between subjective assessments and objective measurement by utilizing machine learning to predict wine quality. By analyzing the physicochemical properties of red wine—such as acidity, alcohol content, and sugar levels—we aim to classify its quality on a scale from 0 to 10.

Using the Red Wine Quality Dataset from the UC Irvine Machine Learning Repository, we evaluate the performance of several classification algorithms: Logistic Regression, Decision Tree, KNN, Naive Bayes and SVM with an RBF kernel. Through hyperparameter tuning and cross-validation, the goal is to identify the most accurate model and demonstrate the practical application of data-driven decision-making in the wine industry [@cortez2009semanticscholar].



## Methods & Results:

### Data Loading


In [ ]:
from IPython.display import Markdown
import pandas as pd

@tbl-wine-dataset-preview provides a preview of the wine dataset used in the analysis.


In [ ]:
#| label: tbl-wine-dataset-preview
#| tbl-cap: Preview of wine dataset to be used in analysis.
df = pd.read_csv("../data/raw/raw_data.csv", sep = ";")
Markdown(df.head().to_markdown(index=False))

### Data Cleaning and Duplicates Handling


In [ ]:
#| label: tbl-missing-values
#| tbl-cap: Summary of Missing Values
missing_values = df.isnull().sum()
missing_values_summary = pd.DataFrame({
    "Column": missing_values.index,
    "Missing Values": missing_values.values
})
Markdown(missing_values_summary.to_markdown(index=False))

A thorough examination of the dataset revealed no missing values in any of the columns. This was verified by checking for null entries in all rows and columns using methods such as `isnull()` and `info()` in Python.


In [ ]:
#| label: tbl-duplicates
#| tbl-cap: Summary of Duplicate Rows
duplicates = df[df.duplicated()]
duplicates_summary = pd.DataFrame({
    "Total Duplicates": [len(duplicates)]
})
Markdown(duplicates_summary.to_markdown(index=False))

# Remove duplicates
df = df.drop_duplicates()

@tbl-missing-values summarizes the missing data in each column.


In [ ]:
#| label: tbl-data-cleaning-summary
#| tbl-cap: Summary of Data Cleaning Steps.
cleaning_summary = pd.DataFrame({
    "Step": ["Initial Missing Values", "Removed Duplicates"],
    "Count": [0, len(duplicates)]
})
cleaning_summary

@tbl-data-cleaning-summary summarizes the data cleaning steps undertaken.

### Columns

- fixed acidity: grams of tartaric acid per cubic decimeter.
- volatile acidity: grams of acetic acid per cubic decimeter.
- citric acid: grams of citric acid per cubic decimeter.
- residual sugar: grams of residual sugar per cubic decimeter.
- chlorides: grams of sodium chloride per cubic decimeter.
- free sulfur dioxide: grams of unreacted sulfur dioxide per cubic decimeter.
- total sulfur dioxide: grams of total sulfur dioxide per cubic decimeter.
- density: density of the wine in grams per cubic decimeter.
- pH: pH value of the wine
- sulphates: grams of potassium sulphate per cubic decimeter
- alcohol : percentage volume of alcohol content.
- quality : integer range from 0 (representing low-quality) to 10 (representing high-quality) [@torok2023ml_wine_quality].


### Data Validation

The dataset has been validated against the following criteria:

1. **Column Data Types**: All columns conform to the expected data types.
2. **Value Ranges**: All columns conform to the specified range. 
3. **Duplicate Rows**: No duplicate rows were found.
4. **Empty Rows**: No null values or rows with all null values were found.

The dataset passed all validation checks successfully.


### Data Splitting

Our dataset was split into two groups - 80% is split for our training data while the remaining 20% is reserved for our testing set. 


### Exploratory Data Analysis (EDA)


In [ ]:
#| label: tbl-x-train-describe
#| tbl-cap: 'Summary of wine data set with count, mean and quartile ranges'
x_train_describe = pd.read_csv("../results/tables/describe_table.csv")
x_train_describe.rename(columns={ x_train_describe.columns[0]: "statistic" }, inplace = True)
Markdown(x_train_describe.to_markdown(index = False))

![Distribution of Target Class in the Data Set](../results/figures/target_distribution_plot.png){#fig-target-dist width=100%}

It is noted that not all possible target classes (from 0 to 10) are present in the training data.

For our Exploratory Data Analysis we wanted to see if our features have a linear correlation with each other. The below plot is a heatmap which shows the pearson correlation for each feature. 

![Wine Quality Features Heatmap - Pearson Correlation](../results/figures/correlation_heatmap.png){#fig-correlation_heatmap width=70%}

It would appear at first glance that there are a few features which seem to have a moderate linear correlation with each other. Some notable connections we can make from @fig-correlation_heatmap is between pH levels and fixed acidity which appears to have a negative correlation. Density and fixed acidity on the other hand appears to have a positive correlation, with fixed acidity being a common feature between both examples. 

![KDE Plots for all features](../results/figures/feature_distributions.png){#fig-feature_distributions width=80%}

The KDE Plot shows that while some of our features are normally distributed, not all of them are. There are a few features which are skewed to the right such as alcohol and sulphates. There are other features in @fig-feature_distributions which have multiple peaks such as volatile acidity and citric acid.

![Regression Pairplot for All Features](../results/figures/feature_pairplots.png){#fig-feature_pairplots width=100%}

Finally @fig-feature_pairplots is a pairplot which shows the actual distribution of the datapoints feature by feature.



### Analysis


#### Model Selection

From the Exploratory Data Analysis, it was observed that all features within the dataset were numeric. In order to ensure the the data is interpreted properly by the models, a preprocessor was used. This preprocessor included a `StandardScaler()` [@scikit_learn_standard_scaler] to ensure standardization of numerical features. 

Several popular classification models were examined for this task which included:.

- Dummy Classifier*
- Decision Tree
- k-Nearest Neightbours Classifier
- Support Vector Machine with Radial Basis Function
- Gaussian Naive Bayes
- Logistic Regression

\* Note this model was used as a baseline, as the default behaviour would always predict the most frequent appearing class in the training set [@scikit_learn_dummy_classifier].

The results of 5-fold cross validation on the training set for each model is shown below:


In [ ]:
#| label: tbl-model-selection-scores
#| tbl-cap: 'Fit time, score time, training score and validation scores for different models.'
model_selection_scores = pd.read_csv("../results/tables/initial_model_scores.csv")
model_selection_scores = model_selection_scores.set_index(model_selection_scores.columns[0])
model_selection_scores.index.name = "model"
Markdown(model_selection_scores.to_markdown(index = True))

From these results, it appeared that the `{python} model_selection_scores["test_score"].idxmax()` gives the best validatio nscores. Therefore, this model will be used for hyperparameter tuning in the following section.



#### Hyperparameter Tuning

After performing `RandomizedSearchCV` on the `{python} model_selection_scores["test_score"].idxmax()` model, the best parameters for the model is as follows in @tbl-best_parameters: 


In [ ]:
#| label: tbl-best_parameters
#| tbl-cap: Best parameters for the SVC model along with the best score.
tuning_score = pd.read_csv("../results/tables/best_params.csv")
Markdown(tuning_score.to_markdown(index = True))

#### Model Evaluation

In [ ]:
test_accuracy_score = pd.read_csv("../results/tables/test_accuracy.csv")
test_accuracy = round(test_accuracy_score.iloc[0][0], 3)

The `{python} model_selection_scores["test_score"].idxmax()` model with the best tuned hyperparameters was used to find accuracy on the testing set. This resulted in an accuracy of `{python} float(test_accuracy)`.

Specific breakdowns of what predictions the model made can be summarized in the confusion matrices below:

![Confusion matrix for predictions where wine quality = 3](../results/figures/confusion_matrix_class_3.png){#fig-confusion_matrix_class_3 width=100%}


![Confusion matrix for predictions where wine quality = 4](../results/figures/confusion_matrix_class_4.png){#fig-confusion_matrix_class_4 width=100%}


![Confusion matrix for predictions where wine quality = 5](../results/figures/confusion_matrix_class_5.png){#fig-confusion_matrix_class_5 width=100%}


![Confusion matrix for predictions where wine quality = 6](../results/figures/confusion_matrix_class_6.png){#fig-confusion_matrix_class_6 width=100%}


![Confusion matrix for predictions where wine quality = 7](../results/figures/confusion_matrix_class_7.png){#fig-confusion_matrix_class_7 width=100%}


![Confusion matrix for predictions where wine quality = 8](../results/figures/confusion_matrix_class_8.png){#fig-confusion_matrix_class_8 width=100%}


The numbers of true positives, true negatives, false positives and false negatives from the above confusion matricies are summarized here:


In [ ]:
#| label: tbl-confusion-matrix-summary
#| tbl-cap: 'Summary of true positives, true negatives, false positives and false negatives from confusion matricies'
confusion_matrix_summary = pd.read_csv("../results/tables/confusion_matrix_summary.csv")
confusion_matrix_summary = confusion_matrix_summary.set_index(confusion_matrix_summary.columns[0])
confusion_matrix_summary.index.name = "Confusion Matrix Metric"
Markdown(confusion_matrix_summary.to_markdown(index = True))

## Discussion

The evaluation of multiple machine learning models for classifying red wine quality revealed that the Support Vector Classifier (SVC) with a Radial Basis Function (RBF) kernel performed the best in terms of validation accuracy after hyperparameter tuning. The final testing accuracy (`{python} float(test_accuracy)`) demonstrated the model's ability to generalize fairly decently to unseen data. Among the other models, Logistic Regression also achieved a reasonable validation accuracy but fell short compared to the SVC RBF. The Decision Tree model exhibited overfitting, achieving perfect accuracy on the training data but only moderate validation accuracy. Models like k-Nearest Neighbors and Naive Bayes performed relatively poorly, with lower accuracies and limited predictive power.

Despite the superior accuracy of the SVC RBF model, it required the longest fit and score times, highlighting potential limitations in computational efficiency, particularly in scenarios requiring real-time predictions or processing large datasets.

The findings are somewhat aligned with expectations. The SVC RBF's strong performance is consistent with its reputation for handling complex, non-linear relationships in the data effectively. However, the overall accuracy `{python} float(test_accuracy)` is lower than ideal for a practical classification system, indicating challenges in predicting wine quality with high precision based solely on the physicochemical features provided. This outcome suggests that wine quality may be influenced by additional factors, such as sensory data or external conditions, that were not captured in the dataset.

The pronounced overfitting in the Decision Tree model and the relatively modest performance of simpler models like Naive Bayes and k-NN were expected, as these models are less equipped to capture intricate relationships in high-dimensional datasets.

The classification accuracy achieved in this project has implications for the practical applications of such models in the wine industry. While the model can provide a rough estimate of wine quality, its predictions may not be reliable enough for high-stakes decisions, such as pricing or marketing. However, it could still serve as a preliminary screening tool for winemakers to assess batches of wine based on their chemical profiles.

The findings also highlight the importance of computational efficiency. Although the SVC RBF model outperformed others in accuracy, its extended fit and score times may limit its usability in time-sensitive applications. This trade-off between accuracy and efficiency should be carefully considered when deploying the model. 

It should be noted that the predictions of targets of this problem have an inherent ordering from 1 to 10, where lower values suggest worse quality wine and higher values suggest better quality wine. 

This study raises several avenues for future exploration: 

* How would including additional physicochemical features or sensory attributes, such as taste or aroma, influence model performance? Similarly, would excluding less impactful features reduce noise and improve accuracy?
* Would increasing the dataset size or balancing the class distribution lead to better generalization performance?
* Will using a different kernel for the SVC yield greater performance?

<br>

## References
